# Climate and Soil

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:


Outputs:




In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

## Soil Data from SoilGrids

250m resolution

        - Bulk Density
        - Cation Exchange Capacity
        - Clay Content
        - Coarse fragments (> 2 mm)
        - Nitrogen
        - Organic Carbon Density
        - Soil Organic Carbon Stock
        - pH
        - Sand Content
        - Soil Organic Carbon
    All averaged from 0-30cm depth and converted to the correct units.

In [ ]:
bdod = ee.Image("projects/soilgrids-isric/bdod_mean").clip(roi) # bulk density
cec = ee.Image("projects/soilgrids-isric/cec_mean") # cation exchange capacity
clay = ee.Image("projects/soilgrids-isric/clay_mean").clip(roi)
cfvo = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(roi) # coarse fragments
nitro = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(roi)
ocd = ee.Image("projects/soilgrids-isric/ocd_mean").clip(roi) # organic carbon density
ocs = ee.Image("projects/soilgrids-isric/ocs_mean").clip(roi) # soil organic carbon stock
phh2o = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(roi)
sand = ee.Image("projects/soilgrids-isric/sand_mean").clip(roi)
soc = ee.Image("projects/soilgrids-isric/soc_mean").clip(roi) # soil organic carbon

# Function to select and calculate weighted mean for specific depth bands
def weighted_means(image, name):
    depths = ['0-5cm', '5-15cm', '15-30cm']
    weights = [1, 2, 3]    
    # Select the bands for each depth
    weighted_bands = image.select([f'.*_{depth}_mean' for depth in depths]).multiply(ee.Image.constant(weights))
    # Sum the weighted bands
    weighted_mean = weighted_bands.reduce(ee.Reducer.sum()).divide(6)
    return weighted_mean

def convert_mass_area(image, name):
    depths = ['0-5cm', '5-15cm', '15-30cm']

    converted_image = image.select([f'.*_{depth}_mean' for depth in depths]).divide()
    # Sum the weighted bands
    weighted_mean = weighted_bands.reduce(ee.Reducer.sum()).divide(6)
    return weighted_mean

# Convert nitrogen and soil organic carbon from g/kg to g/m²
nitro = nitro.divide(100)    # cg/kg to g/kg
soc = soc.divide(10)         # dg/kg to g/kg
nitro = bdod.divide(nitro)



# Apply unit conversions and depth aggregation
bdod = average_depths(bdod, "bdod").divide(100)       # cg/cm³ to g/cm³
cec = average_depths(cec, "cec").divide(10)           # mmol(c)/kg to cmol/kg
clay = average_depths(clay, "clay").divide(10)        # g/kg (‰) to g/100g (%)
cfvo = average_depths(cfvo, "cfvo").divide(10)        # cm3/dm3 (‰) to cm3/100cm3 (%)
ocd = average_depths(ocd, "ocd").divide(10)           # hg/m³ to kg/m³
ocs = ocs.divide(10)                                  # hg/m² to kg/m²
phh2o = average_depths(phh2o, "phh2o").divide(10)     # pH x 10 to pH
sand = average_depths(sand, "sand").divide(10)        # g/kg (‰) to g/100g (%)


# Combine all the soil properties into a single image
soil_properties = bdod.addBands([cec, clay, cfvo, nitro, ocd, ocs, phh2o, sand, soc]).float()

# Export the final image
export_image(soil_properties, "soilgrids", region = roi, scale = 250)

In [10]:
bdod = ee.Image("projects/soilgrids-isric/bdod_mean") # cation exchange capacity

depths = ['0-5cm', '5-15cm', '15-30cm']
weights = [1, 2, 3]
# Select the bands for each depth
weighted_bands = bdod.select([f'.*_{depth}_mean' for depth in depths]).multiply(ee.Image.constant(weights))

map = geemap.Map()
map.addLayer(bdod.select('bdod_5-15cm_mean'), {'min': 100, 'max': 150, 'palette':['blue', 'red']}, 'bdod')
map.addLayer(bdod.select('bdod_15-30cm_mean'), {'min': 100, 'max': 150, 'palette':['blue', 'red']}, 'bdod')
# map.addLayer(weighted_bands, {}, 'weighted_bands')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## TerraClim Data

    Calculated yearly metrics.
        Summed:
        - Solar Radiation
        - Soil Moisture
        - Precipitation
        Averaged:
        - Temperature
        - Vapour Pressure
        - Evapotranspiration

What difference does it make to sum or average?

<!-- # Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality -->

<!-- ## Seasonality index

Walsh and Lawler 1981 -->

In [ ]:
terraclim = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterDate("1985-01-01", "2019-12-31")
    .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort = True, maxPixels = 1024)
    .reproject(crs = "EPSG:4326", scale = 10000))
)

# Function to calculate yearly metrics with scaling applied after filtering
def calculate_yearly_metrics(year):

    def scale_and_aggregate(var, reducer):
        data = terraclim.select(var).filter(ee.Filter.calendarRange(year, year, "year"))
        if var != "pr":
            data = data.map(lambda img: img.multiply(0.1))
        return data.reduce(reducer).toInt16().rename(f"{var}_{year}")

    # Define which variables are processed with sum or mean reducers
    sum_vars = ["srad", "soil", "pr"]
    mean_vars = ["vpd", "aet", "def"]

    # Aggregate sum variables (radiation, soil, precipitation)
    processed_vars = {var: scale_and_aggregate(var, ee.Reducer.sum()) for var in sum_vars}
    
    # Aggregate mean temperature (average of max and min)
    maxtemp = scale_and_aggregate("tmmx", ee.Reducer.mean())
    mintemp = scale_and_aggregate("tmmn", ee.Reducer.mean())
    processed_vars["temp"] = maxtemp.addBands(mintemp).reduce(ee.Reducer.mean()).float().rename(f"temp_{year}")
    
    # Aggregate other mean variables (vpd, aet)
    processed_vars.update({var: scale_and_aggregate(var, ee.Reducer.mean()) for var in mean_vars})

    # Calculate Seasonality Index (SI)
    mean_prec = processed_vars["pr"].divide(12)
    deviations = terraclim.select("pr").filter(ee.Filter.calendarRange(year, year, "year")) \
        .map(lambda img: img.multiply(0.1))\
        .map(lambda month: month.subtract(mean_prec).abs())
    si_band = deviations.reduce(ee.Reducer.sum()).divide(processed_vars["pr"]).float().rename(f"si_{year}")

    return ee.Image.cat([*processed_vars.values(), si_band])

# Create a dictionary for variables with filtering and scaling applied after
vars = {var: terraclim.select(var) for var in ["tmmx", "tmmn", "srad", "vpd", "soil", "aet", "pr"]}

# Calculate yearly metrics and combine into a single image
yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in config.range_1985_2019])

# Function to calculate the mean for a given band pattern
def calculate_mean(band_pattern, new_name):
    return yearly_metrics.select(band_pattern).reduce(ee.Reducer.mean()).rename(new_name)

# Calculate the mean across all years for the desired variables
mean_metrics = {name: calculate_mean(f".*{name}.*", f"mean_{name}") \
                for name in ["pr", "srad", "temp", "vpd", "soil", "aet", "si"]}

# Combine the mean layers into a single image
yearly_terraclim = ee.Image.cat([yearly_metrics, *mean_metrics.values()])

# Export the final image
export_image(yearly_terraclim, "yearly_terraclim", scale = 10000)


## TerraClim future projections

https://gee-community-catalog.org/projects/terraclim/

In [ ]:

# Define the scalor function
def scalor(image):
    scale = ee.Number(image.get('scale'))
    offset = ee.Number(image.get('offset'))
    return image.multiply(scale).add(offset).copyProperties(image, ["system:time_start", "system:time_end"])

# Load the dataset and apply the scalor function
dataset2c = ee.ImageCollection("projects/sat-io/open-datasets/TERRACLIMATE/2C/tmax").select('b10')
dataset2c = dataset2c.map(scalor)

# Load the VPD dataset
vpd_4c = ee.ImageCollection("projects/sat-io/open-datasets/TERRACLIMATE/4C/vpd")



31

<!-- ## Yearly CWD

Calculated as in Malhi et al 2009 and Aragão et al 2007, considering ET fixed as 100mm/month.

[equations](https://imgur.com/o4lVmM7) -->